### Part A

Naive Bayes classifier is an algorithm that uses Bayes' theorem to classify objects. It's called Naive because it makes an assumption that each feature is independent of the other.

In [3]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#Loading the dataset into a pandas data frame
df = pd.read_csv('./adult.csv')
# df.head()
df.isnull().sum().sum()

#Dropping education as educational num cover it
df.drop(["education"], axis = 1,  inplace=True)

#Converting the last column to 0s and 1s
df['income'].replace(['>50K', '<=50K'], [1, 0], inplace=True)
print(df.shape)

# Replacing ? with the most frequent value in each column
df['age'].replace( "?", df['age'].mode()[0], inplace=True)
# print(f"{df['age'].mode()[0]=}")
# type(df['age'].mode())
df['workclass'].replace("?", df['workclass'].mode()[0], inplace=True)
# print(f"{df['workclass'].mode()}")
df['fnlwgt'].replace("?", df['fnlwgt'].mode()[0], inplace=True)
df['educational-num'].replace("?",
                              df['educational-num'].mode()[0], inplace=True)
df['occupation'].replace("?", df['occupation'].mode()[0], inplace=True)
df['race'].replace("?", df['race'].mode()[0], inplace=True)
df['gender'].replace("?", df['gender'].mode()[0], inplace=True)
df['capital-gain'].replace("?", df['capital-gain'].mode()[0], inplace=True)
df['capital-loss'].replace("?", df['capital-loss'].mode()[0], inplace=True)
df['hours-per-week'].replace("?", df['hours-per-week'].mode()[0], inplace=True)
df['native-country'].replace("?", df['native-country'].mode()[0], inplace=True)


#Converting categorical variables to numberic values
df['workclass'].replace(df['workclass'].unique(), [
                        i for i in range(len(df['workclass'].unique()))], inplace=True)
df['marital-status'].replace(df['marital-status'].unique(), [
    i for i in range(len((df['marital-status'].unique())))], inplace=True)
df['relationship'].replace(df['relationship'].unique(), [
    i for i in range(len(df['relationship'].unique()))], inplace=True)
df['gender'].replace(df['gender'].unique(), [
    i for i in range(len(df['gender'].unique()))], inplace=True)
df['native-country'].replace(df['native-country'].unique(), [
    i for i in range(len(df['native-country'].unique()))], inplace=True)
df['occupation'].replace(df['occupation'].unique(), [
    i for i in range(len(df['occupation'].unique()))], inplace=True)
df['race'].replace(df['race'].unique(), [
    i for i in range(len(df['race'].unique()))], inplace=True)

df.head()
df.info()





(48842, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   age              48842 non-null  int64
 1   workclass        48842 non-null  int64
 2   fnlwgt           48842 non-null  int64
 3   educational-num  48842 non-null  int64
 4   marital-status   48842 non-null  int64
 5   occupation       48842 non-null  int64
 6   relationship     48842 non-null  int64
 7   race             48842 non-null  int64
 8   gender           48842 non-null  int64
 9   capital-gain     48842 non-null  int64
 10  capital-loss     48842 non-null  int64
 11  hours-per-week   48842 non-null  int64
 12  native-country   48842 non-null  int64
 13  income           48842 non-null  int64
dtypes: int64(14)
memory usage: 5.2 MB


In [30]:
from sklearn.metrics import f1_score


class NaiveBayes():
    def __init__(self, dataset):
        # Initializing the dataset
        self.dataset = dataset
        self.likelihood = {}
        features = list(self.dataset.columns)[:-1]

        for f in features:
            self.likelihood[f] = {}
            for v in dataset[f].unique():
                self.likelihood[f][v] = {}
                for i in [0, 1]:
                    self.likelihood[f][v][i] = 1e-3


        # Splitting training and testing datasets
        self.training_dataset = self.dataset.iloc[:int(
            0.67*self.dataset.shape[0]), :]
        self.testing_dataset = self.dataset.iloc[int(
            0.33*self.dataset.shape[0]):, :]

        # self.X_train lf.testing_dataset.iloc[:, -1]

        self.prior_probability = []

        self.income_classes = [0, 1]
        for i in self.income_classes:
            # print(f"----------prior = {self.training_dataset[self.training_dataset['income'] == i].shape}-------")
            self.prior_probability.append(len(self.training_dataset[self.training_dataset['income'] == i])/len(self.training_dataset))
            # print(
            #     f"{len(self.training_dataset[self.training_dataset['income'] == i])}")
        
    def calculate_likelihood(self):
        # feature_name = column name
        # feature_val = value needed
        # output_needed = 1 or 0
        features = list(self.training_dataset.columns)[:-1]
        for i in [0, 1]:
            data = self.training_dataset[self.training_dataset["income"] == i]                                         
            for f in features:
                for v in self.training_dataset[f].unique():
                    if v in data[f].values:
                        count = data[f].value_counts()[v] + 1e-3
                    else:
                        count = 1e-3
                    self.likelihood[f][v][i] = count/data.shape[0]

    def naive_bayes(self):
        self.calculate_likelihood()
        testing_data = self.testing_dataset.iloc[:, :-1]
        print(f"{testing_data}")
        features = list(self.testing_dataset.columns)[:-1]
        # We are storing labels as output -> meaning 0 means probability in 0 world and 1 means probability in 1 world
        labels = [0,1]
        # We initialize it to 1
        likelihood_prob = [1e100, 1e100]
        # For every row
        
        for row in testing_data.values:                
                # we iterate through 0 and 1 worlds
                for j in range(len(labels)):
                    # then iterate through all the features and multiply with the likelihood
                    for i in range(len(features)):
                        # print(features[i], row[i])
                        likelihood_prob[j] *= self.likelihood[features[i]][row[i]][j]

        print(f"{self.likelihood=}")
        print(f"{likelihood_prob=}")
        # Calculating posterior probability (only numerator)
        self.posterior_probability = [1, 1]
        for j in range(len(labels)):
            self.posterior_probability[j] = likelihood_prob[j] * self.prior_probability[j]
        print(f"{self.posterior_probability=}")
        y_pred = []
        if self.posterior_probability[0] >= self.posterior_probability[1]:
            y_pred.append(0)
        else:
            y_pred.append(1)
        print(y_pred)
        return np.array(y_pred)


In [31]:
nb = NaiveBayes(df)
# y_test = nb.Y_test
y_pred = nb.naive_bayes()

       age  workclass  fnlwgt  educational-num  marital-status  occupation  \
16117   21          0  187937               10               0           3   
16118   34          0  177437                6               1          11   
16119   32          0  162442               13               0           7   
16120   48          0   83407               14               1           3   
16121   61          0  265201               10               1           3   
...    ...        ...     ...              ...             ...         ...   
48837   27          0  257302               12               1           8   
48838   40          0  154374                9               1           0   
48839   58          0  151910                9               2           6   
48840   22          0  201490                9               0           6   
48841   52          5  287927                9               1           7   

       relationship  race  gender  capital-gain  capital-loss  